In [8]:
import pyaudio
import numpy as np
import wave
CHUNK = 1024
resolution = pyaudio.paInt16 
CHANNELS = 1 
sampling_rate = 16000
record_time=5
Expected_num_of_chunks= int(sampling_rate/CHUNK*record_time) 
p=pyaudio.PyAudio()
stream = p.open(format=resolution, channels=CHANNELS, rate=sampling_rate, input=True, frames_per_buffer=CHUNK)
frames=[]
for i in range(0,Expected_num_of_chunks): 
    data = stream.read(CHUNK)
    frames.append(data)      
array_frames=np.array(frames)
intframes=np.frombuffer(array_frames, np.int16)
stream.stop_stream()
stream.close()
p.terminate()

In [9]:
import IPython.display as ipd
ipd.Audio(intframes,rate=16000)